In [2]:
from sympy import *

Stress boundary condition: Standard constitutive law

In [3]:
SyyBC, Tyy0, Pt0, G, K, dt, Eyyd, div, eta_ve, eta_e = symbols('SyyBC, τyy0, Pt0, G, K, dt, Eyyd, div, ηve, ηe') 
VyN, VyS, dy  = symbols('VyN, VyS, Δy ')
Eyyd = (VyN-VyS)/dy - 1/3*((VyN-VyS)/dy)
Tyy  = 2*eta_ve*(Eyyd + Tyy0/2/eta_e)
Pt   = Pt0 - K*dt*div
Syy  = Tyy - Pt
f    = Syy - SyyBC
julia_code(solve(f, VyN)[0])

'0.25 * (4.0 * VyS .* ηe .* ηve + 3.0 * Δy .* ηe .* (-K .* div .* dt + Pt0 + SyyBC) - 3.0 * Δy .* ηve .* τyy0) ./ (ηe .* ηve)'

Stress boundary condition for the 1D code: generic constitutive law

In [16]:
SyyBC, Tyy0, Pt0, G, K, dt, Eyyd, div, eta_ve, eta_e = symbols('σyyi, τyy0[end], Pt0[end], G, K, Δt, ε̇yy[end], div, ηve[end], ηe[end]') 
VyN, VyS, dy  = symbols('VyN, Vy[end-1], Δy ')
eta_b, eta_s, eta_n = symbols('ηb, ηs, ηn')

# Trial part
Exx_pl, Eyy_pl, Ezz_pl, div_pl = symbols('ε̇xx_pl[end], ε̇yy_pl[end], ε̇zz_pl[end], ∇v_pl[end]')
Exx  = 0.
Eyy  = (VyN-VyS)/dy 
Ezz  = Rational(1,2)*(Exx + Eyy)
div  = Exx + Eyy + Ezz
Exxd = 0.0          - 1/3*div
Eyyd = (VyN-VyS)/dy - 1/3*div
Ezzd = Ezz          - 1/3*div
Tyy  = 2*eta_n*(Eyyd + Tyy0/(2*eta_n) - 0*Eyy_pl) - 2*eta_s*(Exxd - 0*Exx_pl) - 2*eta_s*(Ezzd - 0*Ezz_pl)
Pt   = Pt0 - eta_n*(div - 0*div_pl)
Syy  = Tyy - Pt
f    = Syy - SyyBC
print("Vy[end] = " + julia_code(solve(f, VyN)[0]))

# Corrector part
Exx_pl, Eyy_pl, Ezz_pl, div_pl = symbols('ε̇xx_pl[end], ε̇yy_pl[end], ε̇zz_pl[end], ∇v_pl[end]')
Exx  = 0.
Eyy  = (VyN-VyS)/dy 
Ezz  = Rational(1,2)*(Exx + Eyy)
div  = Exx + Eyy + Ezz
Exxd = 0.0          - 1/3*div
Eyyd = (VyN-VyS)/dy - 1/3*div
Ezzd = Ezz          - 1/3*div
Tyy  = 2*eta_n*(Eyyd + Tyy0/(2*eta_n) - Eyy_pl) - 2*eta_s*(Exxd - Exx_pl) - 2*eta_s*(Ezzd - Ezz_pl)
Pt   = Pt0 - eta_n*(div - div_pl)
Syy  = Tyy - Pt
f    = Syy - SyyBC
print("Vy[end] = " + julia_code(solve(f, VyN)[0]))

Vy[end] = (2.0 * Pt0[end] .* Δy + 5.0 * Vy[end-1] .* ηn + 2.0 * Vy[end-1] .* ηs + 2.0 * Δy .* σyyi - 2.0 * Δy .* τyy0[end]) ./ (5.0 * ηn + 2.0 * ηs)
Vy[end] = (Pt0[end] .* Δy .* ηe[end] + Vy[end-1] .* ηe[end] .^ 2 + Vy[end-1] .* ηe[end] .* ηve[end] - 0.666666666666667 * Δy .* ε̇xx_pl[end] .* ηe[end] .* ηve[end] + 1.33333333333333 * Δy .* ε̇yy_pl[end] .* ηe[end] .* ηve[end] - 0.666666666666667 * Δy .* ε̇zz_pl[end] .* ηe[end] .* ηve[end] + 0.666666666666667 * Δy .* ηe[end] .^ 2 .* ∇v_pl[end] + Δy .* ηe[end] .* σyyi - Δy .* ηve[end] .* τyy0[end]) ./ (ηe[end] .* (ηe[end] + ηve[end]))


In [5]:
Eyy, Exx, Ezz  = symbols('Eyyd, ε̇xxd[end], ε̇zzd[end]')
div = Exx + Eyy + Ezz
Exxd = Exx          - 1/3*div
Eyyd = Eyy - 1/3*div
Ezzd = Ezz          - 1/3*div
Pt  = symbols('Pt[end]')
Tyy  = 4/3*eta_ve*(Eyyd + Tyy0/(4/3*eta_e) - Eyy_pl) - 2/3*eta_ve*(Exxd - Exx_pl) - 2/3*eta_ve*(Ezzd - Ezz_pl)
Ptc  = Pt0 - 2/3*eta_e*(div - div_pl)
Syy  = Tyy - Ptc
f    = Syy - SyyBC
# print("ε̇yyd[end] = " + julia_code(solve(f, Eyyd)[0]))
print("ε̇yy[end] = " + julia_code(solve(f, Eyy)[0]))

ε̇yy[end] = (1.5 * Pt0[end] .* ηe[end] - ε̇xx_pl[end] .* ηe[end] .* ηve[end] - ε̇xxd[end] .* ηe[end] .^ 2 + ε̇xxd[end] .* ηe[end] .* ηve[end] + 2.0 * ε̇yy_pl[end] .* ηe[end] .* ηve[end] - ε̇zz_pl[end] .* ηe[end] .* ηve[end] - ε̇zzd[end] .* ηe[end] .^ 2 + ε̇zzd[end] .* ηe[end] .* ηve[end] + ηe[end] .^ 2 .* ∇v_pl[end] + 1.5 * ηe[end] .* σyyi - 1.5 * ηve[end] .* τyy0[end]) ./ (ηe[end] .* (ηe[end] + 2.0 * ηve[end]))


Stress boundary condition for the 0D code: generic constitutive law

In [15]:
Eyy, Exx, Ezz, Pt0, Tyy0 = symbols('Eyy, ε̇xx, ε̇zz, P0, τyy0')
eta_b, eta_s, eta_n = symbols('ηb, ηs, ηn')
Exx_pl, Eyy_pl, Ezz_pl, div_pl = symbols('ε̇xx_pl, ε̇yy_pl, ε̇zz_pl, ∇v_pl')
Ezz = 1/2*(Exx + Eyy)
div  = 0*Exx + Eyy + Ezz
Exxd = Exx - 1/3*div
Eyyd = Eyy - 1/3*div
Ezzd = Ezz - 1/3*div
Pt  = symbols('Pt')
Tyy  = 2*eta_n*(Eyy + Tyy0/(2*eta_n) - Eyy_pl) - 2*eta_s*(Exx - Exx_pl) - 2*eta_s*(Ezz - Ezz_pl)
Ptc  = Pt0 - eta_b*(div - div_pl)
Syy  = Tyy - Ptc
f    = Syy - SyyBC
# print("ε̇yyd[end] = " + julia_code(solve(f, Eyyd)[0]))
print("ε̇yy = " + julia_code(solve(f, Eyy)[0]))

ε̇yy = (2.0 * P0 - ε̇xx .* ηb + 6.0 * ε̇xx .* ηs - 4.0 * ε̇xx_pl .* ηs + 4.0 * ε̇yy_pl .* ηn - 4.0 * ε̇zz_pl .* ηs + 2.0 * ηb .* ∇v_pl + 2.0 * σyyi - 2.0 * τyy0) ./ (3.0 * ηb + 4.0 * ηn - 2.0 * ηs)
